In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import torch
from transformers import pipeline
import requests
import json
from typing import List, Dict, Tuple
import warnings
warnings.filterwarnings('ignore')

class SimpleRAG:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2",
                 llm_model="mistralai/Mistral-7B-Instruct-v0.2"):
        """
        Инициализация RAG системы

        ⚠️ ЧТО МЕНЯТЬ ЗДЕСЬ:
        - model_name: на другие модели для эмбеддингов (например, "sentence-transformers/all-mpnet-base-v2")
        - llm_model: на другие генеративные модели (например, "google/flan-t5-large", "facebook/bart-large")
        - device_map: изменить на "cuda" если есть GPU, или "cpu" для принудительного использования CPU
        """
        # Модель для создания эмбеддингов
        self.embedding_model = SentenceTransformer(model_name)

        # ⚠️ ЕСЛИ НЕТ GPU ИЛИ МАЛО ПАМЯТИ:
        # self.embedding_model = SentenceTransformer(model_name, device='cpu')

        # Языковая модель для генерации
        # ⚠️ ДЛЯ УПРОЩЕНИЯ ИЛИ ЕСЛИ НЕТ ДОСТУПА К БОЛЬШИМ МОДЕЛЯМ:
        # можно использовать меньшие модели или вообще убрать генерацию
        try:
            self.generator = pipeline(
                "text-generation",
                model=llm_model,
                torch_dtype=torch.float16,  # ⚠️ МЕНЯТЬ НА torch.float32 если нет GPU
                device_map="auto",          # ⚠️ МЕНЯТЬ НА "cpu" если проблемы с GPU
                max_length=512
            )
            self.use_generation = True
        except Exception as e:
            print(f"⚠️ Не удалось загрузить генеративную модель: {e}")
            print("⚠️ Буду использоваться только поиск по документам")
            self.use_generation = False

        # FAISS индекс для поиска
        self.index = None
        self.documents = []

    def load_documents(self, documents: List[str]):
        """
        Загрузка документов в систему

        ⚠️ ЧТО МЕНЯТЬ ЗДЕСЬ:
        - Можно добавить предобработку текста (очистка, нормализация)
        - Можно добавить метаданные для каждого документа
        - Можно добавить чанкинг (разбиение длинных документов)
        """
        # ⚠️ ДОБАВИТЬ ПРЕДОБРАБОТКУ:
        # documents = [self.preprocess_text(doc) for doc in documents]

        # ⚠️ ДОБАВИТЬ ЧАНКИНГ ДЛЯ ДЛИННЫХ ДОКУМЕНТОВ:
        # chunked_docs = []
        # for doc in documents:
        #     chunks = self.chunk_text(doc, chunk_size=500)
        #     chunked_docs.extend(chunks)
        # self.documents = chunked_docs

        self.documents = documents
        print(f"Загружено {len(documents)} документов")

    def preprocess_text(self, text: str) -> str:
        """
        ⚠️ ДОБАВИТЬ ПРЕДОБРАБОТКУ ТЕКСТА:
        - Очистка от HTML тегов
        - Нормализация пробелов
        - Приведение к нижнему регистру (если нужно)
        - Удаление стоп-слов и т.д.
        """
        # Пример предобработки:
        import re
        text = re.sub(r'\s+', ' ', text)  # Удаление лишних пробелов
        text = text.strip()
        # text = text.lower()  # ⚠️ РАСКОММЕНТИРОВАТЬ если нужен нижний регистр
        return text

    def chunk_text(self, text: str, chunk_size: int = 500, overlap: int = 50) -> List[str]:
        """
        ⚠️ ДОБАВИТЬ РАЗБИЕНИЕ ТЕКСТА НА ЧАНКИ:
        - Для длинных документов
        - chunk_size: размер чанка в символах/словах
        - overlap: перекрытие между чанками
        """
        # Простая реализация чанкинга по словам
        words = text.split()
        chunks = []

        for i in range(0, len(words), chunk_size - overlap):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)
            if i + chunk_size >= len(words):
                break

        return chunks

    def build_index(self):
        """Построение поискового индекса"""
        if not self.documents:
            raise ValueError("Сначала загрузите документы")

        # Создаем эмбеддинги для всех документов
        print("Создание эмбеддингов...")
        embeddings = self.embedding_model.encode(self.documents)

        # ⚠️ ВАРИАНТЫ ИНДЕКСОВ FAISS (выбирать в зависимости от размера данных):
        # IndexFlatIP - для маленьких datasets (точный поиск)
        # IndexIVFFlat - для больших datasets (приближенный поиск, быстрее)
        dimension = embeddings.shape[1]

        if len(self.documents) < 10000:
            # Для маленьких коллекций
            self.index = faiss.IndexFlatIP(dimension)
        else:
            # ⚠️ ДЛЯ БОЛЬШИХ КОЛЛЕКЦИЙ (>10к документов):
            quantizer = faiss.IndexFlatIP(dimension)
            nlist = 100  # количество кластеров
            self.index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
            # нужно тренировать индекс
            self.index.train(embeddings)

        # Нормализуем векторы для косинусного сходства
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)

        print(f"Построен индекс с {self.index.ntotal} документами")

    def retrieve(self, query: str, k: int = 3, score_threshold: float = 0.0) -> List[Tuple[str, float]]:
        """
        Поиск релевантных документов

        ⚠️ ЧТО МЕНЯТЬ ЗДЕСЬ:
        - k: количество возвращаемых документов
        - score_threshold: минимальный порог релевантности
        - Можно добавить разные стратегии поиска
        """
        if self.index is None:
            raise ValueError("Сначала постройте индекс")

        # ⚠️ ПРЕДОБРАБОТКА ЗАПРОСА:
        # query = self.preprocess_text(query)

        # Эмбеддинг запроса
        query_embedding = self.embedding_model.encode([query])
        faiss.normalize_L2(query_embedding)

        # Поиск в индексе
        scores, indices = self.index.search(query_embedding, k)

        # Возвращаем документы и оценки
        results = []
        for i, (score, idx) in enumerate(zip(scores[0], indices[0])):
            if idx < len(self.documents) and score >= score_threshold:
                results.append((self.documents[idx], float(score)))

        return results

    def generate_answer(self, query: str, context: List[str]) -> str:
        """
        Генерация ответа на основе контекста

        ⚠️ ЧТО МЕНЯТЬ ЗДЕСЬ:
        - Шаблон промпта (prompt template)
        - Параметры генерации (temperature, max_tokens)
        - Можно добавить несколько шаблонов для разных типов запросов
        """
        if not self.use_generation:
            # ⚠️ ЕСЛИ ГЕНЕРАЦИЯ НЕДОСТУПНА - возвращаем лучший документ
            return f"На основе найденной информации: {context[0][:200]}..." if context else "Информация не найдена"

        # Подготавливаем контекст
        context_text = "\n".join([f"- {doc}" for doc in context])

        # ⚠️ ШАБЛОН ПРОМПТА - МОЖНО ИЗМЕНИТЬ ПОД ЗАДАЧУ:
        prompt = f"""На основе предоставленной информации ответь на вопрос.
Если информации недостаточно, скажи об этом.

Контекст:
{context_text}

Вопрос: {query}

Ответ: """

        # ⚠️ ПАРАМЕТРЫ ГЕНЕРАЦИИ - МОЖНО НАСТРОИТЬ:
        try:
            response = self.generator(
                prompt,
                max_new_tokens=256,      # ⚠️ МЕНЯТЬ: больше/меньше токенов
                do_sample=True,          # ⚠️ МЕНЯТЬ: False для детерминированного вывода
                temperature=0.7,         # ⚠️ МЕНЯТЬ: 0.1-1.0 (меньше = более детерминировано)
                top_p=0.9,               # ⚠️ МЕНЯТЬ: 0.5-1.0 (nucleus sampling)
                pad_token_id=self.generator.tokenizer.eos_token_id
            )

            generated_text = response[0]['generated_text']
            # Извлекаем только ответ (после "Ответ: ")
            if "Ответ: " in generated_text:
                return generated_text.split("Ответ: ")[-1].strip()
            else:
                return generated_text.strip()

        except Exception as e:
            print(f"⚠️ Ошибка генерации: {e}")
            return f"На основе найденной информации: {context[0][:300]}..." if context else "Не удалось сгенерировать ответ"

    def ask(self, query: str, k: int = 3, include_sources: bool = True) -> Dict:
        """
        Полный пайплайн RAG: поиск + генерация

        ⚠️ ЧТО МЕНЯТЬ ЗДЕСЬ:
        - k: количество документов для поиска
        - include_sources: включать ли исходные документы в ответ
        - Можно добавить фильтрацию по времени, источнику и т.д.
        """
        # Поиск релевантных документов
        retrieved_docs = self.retrieve(query, k)

        if not retrieved_docs:
            return {
                "answer": "Не найдено релевантной информации",
                "sources": [],
                "query": query
            }

        # Извлекаем только тексты документов
        context_texts = [doc for doc, score in retrieved_docs]

        # Генерация ответа
        answer = self.generate_answer(query, context_texts)

        result = {
            "answer": answer,
            "query": query
        }

        if include_sources:
            result["sources"] = retrieved_docs

        return result

# ⚠️ ФУНКЦИЯ ДЛЯ ЗАГРУЗКИ РАЗНЫХ ТИПОВ ДАННЫХ
def load_data_from_source(source_type: str, source_path: str) -> List[str]:
    """
    Загрузка данных из различных источников

    ⚠️ ДОБАВИТЬ ПОДДЕРЖКУ НУЖНЫХ ФОРМАТОВ:
    - txt, csv, json, pdf, docx, базы данных и т.д.
    """
    documents = []

    if source_type == "txt":
        # Загрузка из текстового файла
        try:
            with open(source_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # ⚠️ НАСТРОИТЬ РАЗДЕЛЕНИЕ ДОКУМЕНТОВ:
                documents = [doc.strip() for doc in content.split('\n\n') if doc.strip()]
        except Exception as e:
            print(f"Ошибка загрузки файла {source_path}: {e}")

    elif source_type == "csv":
        # ⚠️ ДОБАВИТЬ ЗАГРУЗКУ ИЗ CSV:
        try:
            df = pd.read_csv(source_path)
            # Выбираем колонку с текстом
            if 'text' in df.columns:
                documents = df['text'].dropna().tolist()
        except Exception as e:
            print(f"Ошибка загрузки CSV {source_path}: {e}")

    elif source_type == "json":
        # ⚠️ ДОБАВИТЬ ЗАГРУЗКУ ИЗ JSON:
        try:
            with open(source_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                # Настройка в зависимости от структуры JSON
                if isinstance(data, list):
                    documents = [str(item) for item in data]
        except Exception as e:
            print(f"Ошибка загрузки JSON {source_path}: {e}")

    else:
        print(f"⚠️ Неподдерживаемый тип источника: {source_type}")

    return documents

# ⚠️ КОНФИГУРАЦИЯ - МЕНЯТЬ ПАРАМЕТРЫ ЗДЕСЬ
class RAGConfig:
    """Конфигурация RAG системы"""

    # ⚠️ МОДЕЛИ
    EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # Альтернативы: "all-mpnet-base-v2"
    LLM_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"           # Альтернативы: "facebook/bart-large"

    # ⚠️ ПАРАМЕТРЫ ПОИСКА
    TOP_K = 3                   # Количество возвращаемых документов
    SCORE_THRESHOLD = 0.0       # Минимальный порог релевантности

    # ⚠️ ПАРАМЕТРЫ ГЕНЕРАЦИИ
    MAX_NEW_TOKENS = 256
    TEMPERATURE = 0.7
    DO_SAMPLE = True

    # ⚠️ ПАРАМЕТРЫ ДАННЫХ
    CHUNK_SIZE = 500            # Для чанкинга
    CHUNK_OVERLAP = 50          # Перекрытие чанков

def main():
    """
    ⚠️ ОСНОВНАЯ ФУНКЦИЯ - МЕНЯТЬ ПОД СВОИ ДАННЫЕ И ЗАДАЧИ
    """

    # 1. ⚠️ ИНИЦИАЛИЗАЦИЯ С КОНФИГОМ
    rag_system = SimpleRAG(
        model_name=RAGConfig.EMBEDDING_MODEL,
        llm_model=RAGConfig.LLM_MODEL
    )

    # 2. ⚠️ ЗАГРУЗКА ДАННЫХ - ВЫБРАТЬ НУЖНЫЙ СПОСОБ:

    # Способ A: Прямая загрузка списка текстов
    documents = [
        "Искусственный интеллект - это область компьютерных наук...",
        "Машинное обучение является подразделом искусственного интеллекта...",
        # ⚠️ ДОБАВИТЬ СВОИ ДОКУМЕНТЫ ЗДЕСЬ
    ]

    # Способ B: Загрузка из файла
    # documents = load_data_from_source("txt", "data/my_documents.txt")
    # documents = load_data_from_source("csv", "data/documents.csv")
    # documents = load_data_from_source("json", "data/documents.json")

    # Способ C: Загрузка из базы данных или API
    # documents = load_from_database()  # ⚠️ РЕАЛИЗОВАТЬ ФУНКЦИЮ

    rag_system.load_documents(documents)
    rag_system.build_index()

    # 3. ⚠️ ТЕСТИРОВАНИЕ - ДОБАВИТЬ СВОИ ЗАПРОСЫ
    test_queries = [
        "Что такое искусственный интеллект?",
        "Какие есть виды машинного обучения?",
        # ⚠️ ДОБАВИТЬ СВОИ ТЕСТОВЫЕ ЗАПРОСЫ
    ]

    print("=" * 60)
    print("ТЕСТИРОВАНИЕ RAG СИСТЕМЫ")
    print("=" * 60)

    for query in test_queries:
        print(f"\n📝 Вопрос: {query}")
        result = rag_system.ask(query, k=RAGConfig.TOP_K)

        print(f"🤖 Ответ: {result['answer']}")
        if 'sources' in result:
            print("📚 Источники:")
            for i, (source, score) in enumerate(result['sources']):
                print(f"   {i+1}. [сходство: {score:.3f}] {source[:100]}...")
        print("-" * 60)

    # 4. ⚠️ ГЕНЕРАЦИЯ САБМИТА - НАСТРОИТЬ ПОД ФОРМАТ ТРЕБОВАНИЙ
    generate_submission(
        test_queries,
        rag_system,
        output_file="my_rag_predictions.json"
    )

def generate_submission(test_queries: List[str], rag_system: SimpleRAG,
                       output_file: str = "rag_predictions.json"):
    """
    Генерация сабмита для тестовых запросов

    ⚠️ НАСТРОИТЬ ФОРМАТ ВЫВОДА ПОД ТРЕБОВАНИЯ:
    - Структура JSON
    - Включаемые поля
    - Форматирование ответов
    """
    predictions = []

    for i, query in enumerate(test_queries):
        print(f"Обработка запроса {i+1}/{len(test_queries)}: {query}")

        result = rag_system.ask(query, k=RAGConfig.TOP_K)

        # ⚠️ НАСТРОИТЬ СТРУКТУРУ САБМИТА:
        prediction = {
            "id": i + 1,                    # ⚠️ МОЖЕТ БЫТЬ ДРУГОЙ ID
            "query": query,
            "answer": result["answer"],
            "sources": [
                {
                    "text": doc[:200] + "...",  # ⚠️ НАСТРОИТЬ ДЛИНУ ТЕКСТА
                    "score": float(score),
                    "source_id": j            # ⚠️ ДОБАВИТЬ ID ИСТОЧНИКА ЕСЛИ ЕСТЬ
                }
                for j, (doc, score) in enumerate(result.get("sources", []))
            ],
            "timestamp": pd.Timestamp.now().isoformat()  # ⚠️ ОПЦИОНАЛЬНО
        }

        predictions.append(prediction)

    # ⚠️ НАСТРОИТЬ ФОРМАТ СОХРАНЕНИЯ:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(predictions, f, ensure_ascii=False, indent=2)

    print(f"✅ Сабмит сохранен в файл: {output_file}")
    print(f"📊 Обработано запросов: {len(predictions)}")

    return predictions

if __name__ == "__main__":
    # ⚠️ ВКЛЮЧИТЬ/ВЫКЛЮЧИТЬ РАЗНЫЕ ЧАСТИ:

    # Запуск полной демонстрации
    main()

    # Или только тестирование
    # test_my_rag()

    # Или только генерация сабмита
    # generate_my_submission()